Bagging + XGBClassifier

In [7]:
import pickle
import pandas as pd
from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
import xgboost

In [2]:
df = pd.read_csv("../aus.csv")
df.head(10)

,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,month,day
0,2,13.4,22.9,0.6,13,44.0,13,14,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8,0,0,12,1
1,2,7.4,25.1,0.0,14,44.0,6,15,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3,0,0,12,2
2,2,12.9,25.7,0.0,15,46.0,13,15,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2,0,0,12,3
3,2,9.2,28.0,0.0,4,24.0,9,0,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5,0,0,12,4
4,2,17.5,32.3,1.0,13,41.0,1,7,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7,0,0,12,5
5,2,14.6,29.7,0.2,14,56.0,13,13,19.0,24.0,55.0,23.0,1009.2,1005.4,20.6,28.9,0,0,12,6
6,2,14.3,25.0,0.0,13,50.0,12,13,20.0,24.0,49.0,19.0,1009.6,1008.2,18.1,24.6,0,0,12,7
7,2,7.7,26.7,0.0,13,35.0,10,13,6.0,17.0,48.0,19.0,1013.4,1010.1,16.3,25.5,0,0,12,8
8,2,9.7,31.9,0.0,6,80.0,9,7,7.0,28.0,42.0,9.0,1008.9,1003.6,18.3,30.2,0,1,12,9
9,2,13.1,30.1,1.4,13,28.0,8,10,15.0,11.0,58.0,27.0,1007.0,1005.7,20.1,28.2,1,0,12,10


In [3]:
y = df['RainTomorrow']
x = df.drop(['RainTomorrow'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=True, random_state=66)
y.value_counts()

RainTomorrow
0    93693
1    17504
Name: count, dtype: int64

In [8]:
xgb = xgboost.XGBClassifier()
estimators = [
     ('lr', LogisticRegression(max_iter=1000))]
clf = StackingClassifier(estimators,xgb)

In [9]:
clf.fit(x_train, y_train)

StackingClassifier(estimators=[('lr', LogisticRegression(max_iter=1000))],
                   final_estimator=XGBClassifier(base_score=None, booster=None,
                                                 callbacks=None,
                                                 colsample_bylevel=None,
                                                 colsample_bynode=None,
                                                 colsample_bytree=None,
                                                 device=None,
                                                 early_stopping_rounds=None,
                                                 enable_categorical=False,
                                                 eval_metric=None,
                                                 feature_types=None, gamma=None,
                                                 grow_policy=None,
                                                 importance_type=None,
                                                 interaction_constraints=None,
                                                 learning_rate=None,
                                                 max_bin=None,
                                                 max_cat_threshold=None,
                                                 max_cat_to_onehot=None,
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 max_leaves=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 multi_strategy=None,
                                                 n_estimators=None, n_jobs=None,
                                                 num_parallel_tree=None,
                                                 random_state=None, ...))

In [10]:
prediction = clf.predict(x_test)

In [11]:
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.88      0.98      0.93     28112
           1       0.71      0.27      0.39      5248

    accuracy                           0.87     33360
   macro avg       0.79      0.62      0.66     33360
weighted avg       0.85      0.87      0.84     33360



In [12]:
pickle.dump(clf, open('stacking.pkl', 'wb'))